In [5]:
from pathlib import Path
import pandas as pd

import starfile
import dynamotable


In [3]:
working_directory = Path("/mnt/scratch/ribosomes/kas_k44a/old_relion/")
input_starfile = {}

optics_columns = [
    "rlnOpticsGroup",
    "rlnOpticsGroupName",
    "rlnSphericalAberration",
    "rlnVoltage",
    "rlnTomoTiltSeriesPixelSize",
    "rlnCtfDataAreCtfPremultiplied",
    "rlnImageDimensionality",
    "rlnTomoSubtomogramBinning",
    "rlnImagePixelSize",
    "rlnImageSize",
]
input_starfile["optics"] = pd.DataFrame(columns=optics_columns) 
input_starfile["optics"].loc[len(input_starfile["optics"])] = [1, "opticsGroup1", 2.7, 300, 1.0825, 1, 3, 8, 8.66, 64]
input_starfile["particles"] = starfile.read("/mnt/scratch/ribosomes/kas_k44a/eman2/inverted_seg_particles/particles_abs0.1rel0.1.star")
input_starfile["particles"].rename(columns={"rlnOriginX": "rlnOriginXAngst", "rlnOriginY": "rlnOriginYAngst", "rlnOriginZ": "rlnOriginZAngst", "rlnMicrographName" : "rlnTomoName"}, inplace=True)
input_starfile["particles"]["rlnTomoName"] = input_starfile["particles"]["rlnTomoName"].str.replace(".mrc", "")
#input_starfile["particles"].drop(columns=["rlnPixelSize"], inplace=True)

input_starfile["particles"]["rlnOpticsGroup"] = 1

In [4]:
input_starfile["particles"].sort_values(
    by=["rlnTomoName", "rlnCoordinateX", "rlnCoordinateY","rlnCoordinateZ"], 
    inplace=True, 
    ignore_index=True
    )
input_starfile["particles"].head()


,rlnCoordinateZ,rlnCoordinateY,rlnCoordinateX,rlnAngleRot,rlnAngleTilt,rlnAnglePsi,rlnTomoName,rlnOpticsGroup
0,2730,5180,580,53,114,3,ts_001_b10,1
1,2640,4880,600,63,135,7,ts_001_b10,1
2,2040,8480,600,44,76,4,ts_001_b10,1
3,2310,4420,620,134,109,83,ts_001_b10,1
4,2760,10680,700,58,78,113,ts_001_b10,1


In [48]:
for rlnTomoName in input_starfile["particles"]["rlnTomoName"].unique():
    output_starfile = input_starfile.copy()
    output_starfile["particles"] = output_starfile["particles"][
        output_starfile["particles"]["rlnTomoName"] == rlnTomoName
    ]
    output_starfile["particles"]["rlnCoordinateZ"] = output_starfile["particles"]["rlnCoordinateZ"] - 100
    starfile.write(output_starfile, working_directory / f"{rlnTomoName.split('.')[0]}_particles.star", overwrite=True)

/tmp/ipykernel_416344/1926229425.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_starfile["particles"]["rlnCoordinateZ"] = output_starfile["particles"]["rlnCoordinateZ"] - 100


In [57]:
ts001 = starfile.read(working_directory / "run_data.star")
ts001["particles"] = ts001["particles"][ts001["particles"]["rlnMicrographName"] == "/data/michalakdj/202206_kas_krios/frames/reconstruction/AAA1_tilt030.mrc_8.66Apx.mrc"]

ts001["particles"]["rlnCoordinateX"] = ts001["particles"]["rlnCoordinateX"] - 100 * (8.66 / 3.0)
ts001["particles"]["rlnCoordinateY"] = ts001["particles"]["rlnCoordinateY"] - 100 * (8.66 / 3.0)
ts001["particles"]["rlnCoordinateZ"] = ts001["particles"]["rlnCoordinateZ"] - 100 * (8.66 / 3.0)
starfile.write(ts001, working_directory / "run_data_ts001.star", overwrite=True)

In [53]:
len(ts001["particles"].value_counts())

207

In [58]:
old_relion_star = starfile.read("/mnt/scratch/ribosomes/kas_k44a/old_relion/run_data.star")

ts_001_df = old_relion_star.copy()
ts_001_df["optics"]["rlnTomoTiltSeriesPixelSize"] = 3

ts_001_df["particles"] = old_relion_star["particles"][old_relion_star["particles"]["rlnMicrographName"] == '/data/michalakdj/202206_kas_krios/frames/reconstruction/AAA1_tilt030.mrc_8.66Apx.mrc']
ts_001_df["particles"]["rlnCoordinateX"] = ts_001_df["particles"]["rlnCoordinateX"] * (3/10.825)
ts_001_df["particles"]["rlnCoordinateY"] = ts_001_df["particles"]["rlnCoordinateY"] * (3/10.825)
ts_001_df["particles"]["rlnCoordinateZ"] = ts_001_df["particles"]["rlnCoordinateZ"] * (3/10.825) - 40
ts_001_df["particles"]["rlnTomoName"] = "ts_001"

/tmp/ipykernel_500276/3578741538.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ts_001_df["particles"]["rlnCoordinateX"] = ts_001_df["particles"]["rlnCoordinateX"] * (3/10.825)
/tmp/ipykernel_500276/3578741538.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ts_001_df["particles"]["rlnCoordinateY"] = ts_001_df["particles"]["rlnCoordinateY"] * (3/10.825)
/tmp/ipykernel_500276/3578741538.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

In [59]:
starfile.write(ts_001_df, "/mnt/scratch/ribosomes/kas_k44a/old_relion/ts_001.star", overwrite=True)

In [49]:

old_relion_star["particles"]["rlnMicrographName"][0]

'/data/michalakdj/202206_kas_krios/frames/reconstruction/AAA1_tilt030.mrc_8.66Apx.mrc'